In [1]:
import pandas as pd
import sklearn as skl
combats = pd.read_csv("combats.csv", delimiter=',')
pokemon = pd.read_csv("pokemon.csv", delimiter=',')
pokemon_id_each_team = pd.read_csv("pokemon_id_each_team.csv", delimiter=',')
team_combat = pd.read_csv("team_combat.csv")
pokemon = pokemon.drop(['HP', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Name', 'Type 1', 'Type 2', 'Legendary'], axis = 1)
pokemon = pokemon.rename(columns={'#':'id'})
pokemon_id_each_team = pokemon_id_each_team.rename(columns={'#':'id'})

In [40]:
#combats.describe()
#pokemon.describe()
#pokemon_id_each_team.describe()
#team_combat.describe()

In [7]:
# Датафрейм с информацией об одиночных победах покемонов
wins = pd.DataFrame({'id': range(1, 801), 'wins': 0, 'battles': 0})
for i in range(len(combats)):
    winner = combats.iloc[i]['Winner']
    wins.iloc[winner-1]['wins'] += 1
    tmp_pokemon1 = combats.iloc[i]['First_pokemon']
    wins.iloc[tmp_pokemon1-1]['battles'] += 1
    tmp_pokemon2 = combats.iloc[i]['Second_pokemon']
    wins.iloc[tmp_pokemon2-1]['battles'] += 1
wins['win_pct'] = wins['wins'] / wins['battles']

In [20]:
# Датафрейм с информацией о победах покемонов в командах
team_wins = pd.DataFrame({'id': range(1, 801), 'wins': 0, 'battles': 0})
for i in range(len(team_combat)):
    winner = team_combat.iloc[i]['winner']
    for j in range(6):
        tmp_pokemon1 = pokemon_id_each_team.iloc[team_combat.iloc[i][winner]-1][j+1]
        tmp_pokemon2 = pokemon_id_each_team.iloc[team_combat.iloc[i][(winner+1)%2]-1][j+1]
        team_wins.iloc[tmp_pokemon1-1]['battles'] += 1
        team_wins.iloc[tmp_pokemon2-1]['battles'] += 1
        team_wins.iloc[tmp_pokemon1-1]['wins'] += 1
team_wins['win_pct'] = team_wins['wins'] / team_wins['battles']

In [39]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
df = pokemon.merge(wins, on='id').merge(team_wins, on='id')
df['win_pct_y'].fillna(value=0.5, inplace=True)
#df = df.dropna(subset=['win_pct_y'])
df = df.dropna(subset=['win_pct_x'])
X = df[['win_pct_x', 'win_pct_y']]
y = df['Attack']
lr = LinearRegression()
cv = KFold(n_splits=10, shuffle=True, random_state=0)
scores = cross_val_score(lr,X,y, cv=cv)
print(scores.mean())

0.23993760513148144
